개별 학교 전교생 수 예측


1.   각 학교와 출생아 수 상관관계 파악 -> 상관관계가 높으면 회귀로, 적으면 따로 분석
2.   상관관계가 낮은 학교들은



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#1. 전체 출생아수(X), 전교생수(Y)로 할 때, 회귀모델 -> P, R값도 추가

In [3]:
#1. 전체 전교생 예측

In [4]:
#1) 출생아수 데이터 전처리
import pandas as pd

file_path = "/content/drive/MyDrive/Colab Notebooks/DSL/EDA2_Education/data/출생아수.csv"
df_bir = pd.read_csv(file_path, encoding='utf-8-sig')

In [5]:
df_bir = df_bir.iloc[[0]]
df_bir = df_bir.set_index("시군구별")
df_bir = df_bir.apply(pd.to_numeric, errors='coerce')

In [6]:
df_bir.head()

,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
시군구별,,,,,,,,,,,,,,,,,,,,,
전국,675394,641594,620668,640089,559934,496911,495036,476958,438707,451759,...,435435,438420,406243,357771,326822,302676,272337,260562,249186,230028


In [7]:
# df_birth 생성
start_years = range(1997, 2019)  # 1997~2018
end_years = range(2002, 2024)    # 2002~2023
new_years = range(2009, 2031)    # 2009~2030

df_birth = pd.DataFrame()

for start, end, new_col in zip(start_years, end_years, new_years):
    cols = [str(year) for year in range(start, end + 1) if str(year) in df_bir.columns]
    df_birth[str(new_col)] = df_bir[cols].sum(axis=1)

In [8]:
#2) 학교 전교생수 전처리
file_path = "/content/drive/MyDrive/Colab Notebooks/DSL/EDA2_Education/data/유초중등학교개황(09-23).csv"
df_students = pd.read_csv(file_path, encoding='euc-kr')

In [9]:
df_students.head()

,조사년도,개방ID,학교명,학교급명,시도명,학제유형명,유초중등학교개황_1학년주간여학생수,유초중등학교개황_1학년주간학급수,유초중등학교개황_순회야간학급수,유초중등학교개황_순회주간학급수,...,유초중등학교개황_3학년주간학급수,유초중등학교개황_2학년야간학생수,유초중등학교개황_2학년야간여학생수,유초중등학교개황_2학년야간학급수,유초중등학교개황_2학년주간학생수,유초중등학교개황_2학년주간여학생수,유초중등학교개황_2학년주간학급수,유초중등학교개황_1학년야간학생수,유초중등학교개황_1학년야간여학생수,유초중등학교개황_1학년야간학급수
0,2009,1000270366,전주초포초등학교병설유치원,원천코드없음,전북,유치원,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2009,1000279486,서울봉현초등학교,원천코드없음,서울,초등학교,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2009,1001306772,인천소방고등학교,원천코드없음,인천,전문계고등학교,0,14,0,0,...,14,0,0,0,414,0,14,0,0,0
3,2009,1001514911,여수구봉초등학교병설유치원,원천코드없음,전남,유치원,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2009,1001970350,산남중학교,원천코드없음,충북,중학교,142,8,0,0,...,8,0,0,0,289,143,8,0,0,0


In [10]:
columns_to_keep = [
    '조사년도', '시도명', '학제유형명', '학교명', '개방ID', '유초중등학교개황_학생수'
]
df_students = df_students[columns_to_keep]
#초등학교 data만 추출(중고등은 이름이 다양한 것에 주의!)
df_students = df_students[df_students['학제유형명']=='초등학교']

# 학생수 피벗 테이블
pivot_students = pd.pivot_table(
    df_students,
    index=['시도명', '학제유형명', '개방ID', '학교명'],  # 행
    columns='조사년도',  # 열
    values='유초중등학교개황_학생수',  # 값
    aggfunc='sum'
)

#결측값 0으로
pivot_students = pivot_students.fillna(0)

# 폐교된 학교 , 이름 바뀐 학교 -> 최신의 이름으로 데이터 올라와 있음, 휴교학교 -> 존재
# 따라서 총학생수 0직전의 전교생수가 30이상인 것만 수동으로 search

In [11]:
#학생수가 0직전 30이상인 학교들만 추출
students_zero_list = pivot_students[pivot_students[2023]==0]
# 연도 컬럼 목록 추출 (역순)
year_columns = list(map(int, range(2023, 2008, -1)))

# 1. 연도 값이 0인 부분 찾기
mask_zero = (students_zero_list[year_columns] == 0).cumprod(axis=1).astype(bool)

# 2. 0이 아닌 값이 처음 나타나는 곳 찾기
first_nonzero = (~mask_zero).idxmax(axis=1)

# 3. 해당 위치의 값이 30 이상인 행 필터링
valid_rows = students_zero_list.apply(lambda row: row[first_nonzero[row.name]] >= 30, axis=1)
filtered_df = students_zero_list[valid_rows]

#휴교학교의 수는 10개정도 있고, 그 전교생 수들이 주변 학교로 이전되기 때문에 휴교학교들도 폐교라 생각하고 예측하기로 함.

In [12]:
filtered_df

조사년도                                   2009   2010   2011   2012   2013  \
시도명 학제유형명 개방ID       학교명                                                  
강원  초등학교  3829408109 옥천초등학교운산분교장        4.0    6.0    7.0    6.0    5.0   
          5036247134 사음초등학교            95.0   79.0   88.0   78.0    0.0   
경기  초등학교  2207679629 서종초등학교정배분교장(폐교)   66.0   72.0   86.0   89.0   92.0   
          2562967717 금주초등학교            77.0   71.0   60.0   51.0   45.0   
          4981731772 법원초등학교            99.0   99.0   91.0   94.0   95.0   
...                                     ...    ...    ...    ...    ...   
충북  초등학교  4095767021 내곡초등학교            55.0   59.0   67.0   64.0   61.0   
          7567409157 옥산초등학교소로분교장       24.0   21.0   50.0   56.0   40.0   
          7974374294 상신초등학교            55.0   65.0   62.0   65.0   55.0   
          8923405289 대장초등학교            57.0   51.0   40.0   35.0   42.0   
          9132677137 남한강초등학교          516.0  437.0  400.0  361.0  331.0   

조사년도                                   2014   2015   2016   2017   2018  \
시도명 학제유형명 개방ID       학교명                                                  
강원  초등학교  3829408109 옥천초등학교운산분교장        5.0    5.0    6.0    8.0   28.0   
          5036247134 사음초등학교             0.0    0.0    0.0    0.0    0.0   
경기  초등학교  2207679629 서종초등학교정배분교장(폐교)    0.0    0.0    0.0    0.0    0.0   
          2562967717 금주초등학교            44.0   42.0   40.0   33.0   36.0   
          4981731772 법원초등학교            88.0   85.0   67.0   57.0   52.0   
...                                     ...    ...    ...    ...    ...   
충북  초등학교  4095767021 내곡초등학교            57.0   55.0   56.0   44.0   50.0   
          7567409157 옥산초등학교소로분교장       36.0   44.0   32.0   39.0   39.0   
          7974374294 상신초등학교            48.0   43.0   46.0   43.0   32.0   
          8923405289 대장초등학교            36.0   29.0   28.0   36.0   35.0   
          9132677137 남한강초등학교          306.0  284.0  283.0  242.0  217.0   

조사년도                                   2019  2020  2021  2022  2023  
시도명 학제유형명 개방ID       학교명                                             
강원  초등학교  3829408109 옥천초등학교운산분교장       39.0  43.0  54.0   0.0   0.0  
          5036247134 사음초등학교             0.0   0.0   0.0   0.0   0.0  
경기  초등학교  2207679629 서종초등학교정배분교장(폐교)    0.0   0.0   0.0   0.0   0.0  
          2562967717 금주초등학교            34.0  32.0  32.0   0.0   0.0  
          4981731772 법원초등학교             0.0   0.0   0.0   0.0   0.0  
...                                     ...   ...   ...   ...   ...  
충북  초등학교  4095767021 내곡초등학교             0.0   0.0   0.0   0.0   0.0  
          7567409157 옥산초등학교소로분교장        0.0   0.0   0.0   0.0   0.0  
          7974374294 상신초등학교             0.0   0.0   0.0   0.0   0.0  
          8923405289 대장초등학교             0.0   0.0   0.0   0.0   0.0  
          9132677137 남한강초등학교          214.0   0.0   0.0   0.0   0.0  

[85 rows x 15 columns]

In [13]:
# 2023년 전교생수가 0인 값 제거
pivot_students = pivot_students[pivot_students[2023] != 0]

In [14]:
pivot_students

조사년도                           2009    2010    2011    2012    2013    2014  \
시도명 학제유형명 개방ID       학교명                                                      
강원  초등학교  1008023023 섬강초등학교     0.0     0.0     0.0     0.0     0.0     0.0   
          1059736549 화동초등학교    58.0    54.0    50.0    44.0    38.0    32.0   
          1110422253 정라초등학교  1269.0  1257.0  1213.0  1164.0  1078.0  1024.0   
          1112041771 성림초등학교  1292.0  1314.0  1311.0  1349.0  1299.0  1267.0   
          1128984749 속사초등학교    20.0    21.0    23.0    22.0    25.0    24.0   
...                             ...     ...     ...     ...     ...     ...   
충북  초등학교  9767946541 삼성초등학교   403.0   345.0   307.0   294.0   277.0   258.0   
          9770097672 오석초등학교    66.0    62.0    61.0    80.0    70.0    64.0   
          9819334983 옥산초등학교   777.0   726.0   665.0   607.0   556.0   530.0   
          9872796773 분평초등학교  1054.0   986.0   915.0   823.0   729.0   698.0   
          9935361981 장연초등학교    58.0    51.0    47.0    41.0    37.0    34.0   

조사년도                           2015    2016    2017    2018    2019    2020  \
시도명 학제유형명 개방ID       학교명                                                      
강원  초등학교  1008023023 섬강초등학교     0.0     0.0     0.0     0.0   956.0  1568.0   
          1059736549 화동초등학교    34.0    43.0    44.0    44.0    41.0    35.0   
          1110422253 정라초등학교   959.0   898.0   840.0   759.0   709.0   678.0   
          1112041771 성림초등학교  1202.0  1151.0  1146.0  1115.0  1141.0  1152.0   
          1128984749 속사초등학교    21.0    27.0    34.0    35.0    49.0    45.0   
...                             ...     ...     ...     ...     ...     ...   
충북  초등학교  9767946541 삼성초등학교   236.0   231.0   220.0   204.0   177.0   164.0   
          9770097672 오석초등학교    59.0    56.0    56.0    60.0    61.0    50.0   
          9819334983 옥산초등학교   518.0   497.0   575.0   595.0   473.0   378.0   
          9872796773 분평초등학교   661.0   629.0   603.0   569.0   530.0   498.0   
          9935361981 장연초등학교    24.0    19.0    16.0    16.0    13.0    10.0   

조사년도                           2021    2022    2023  
시도명 학제유형명 개방ID       학교명                             
강원  초등학교  1008023023 섬강초등학교  1242.0  1299.0  1360.0  
          1059736549 화동초등학교    31.0    24.0    18.0  
          1110422253 정라초등학교   630.0   567.0   512.0  
          1112041771 성림초등학교  1029.0   942.0   892.0  
          1128984749 속사초등학교    50.0    48.0    37.0  
...                             ...     ...     ...  
충북  초등학교  9767946541 삼성초등학교   154.0   149.0   139.0  
          9770097672 오석초등학교    35.0    31.0    32.0  
          9819334983 옥산초등학교   340.0   338.0   306.0  
          9872796773 분평초등학교   481.0   466.0   430.0  
          9935361981 장연초등학교    39.0    41.0    35.0  

[6288 rows x 15 columns]

In [15]:
#3) 폐교리스트 전처리

In [16]:
file_path = "/content/drive/MyDrive/Colab Notebooks/DSL/EDA2_Education/data/시도교육청_폐교목록_전국단위.csv"
df_closed = pd.read_csv(file_path, encoding='utf-8-sig')

In [17]:
df_closed["폐교연도"] = df_closed["폐교연도"].astype(int)

# 2009년 이상 폐교 + 급별이 '초'인 데이터 필터링
df_closed = df_closed[(df_closed["폐교연도"] >= 2009) & (df_closed["급별"] == "초")]

# 불필요한 열(지역교육청, 활용현황, 주소) 제거
df_closed = df_closed.drop(columns=["지역교육청", "활용현황", "주소"])

#폐교리스트가 안맞는게 많음

In [18]:
df_closed

,시도교육청,폐교명,폐교연도,급별
4,경기,(구) 삼송초,2014,초
5,경기,(구) 상신초,2017,초
13,충북,(구)내북초,2013,초
18,부산,(구)동천초등학교,2010,초
19,경기,(구)동패초,2020,초
...,...,...,...,...
3917,부산,회동초등학교,2018,초
3918,전북,회룡초,2012,초
3930,강원,횡성초 당평분교장,2016,초
3934,울산,효문분교,2020,초


In [19]:
#폐교학교 리스트에 폐교가 되지 않은 학교도 존재.
#가정이 필요할듯 -> 전교생 5명이어도 폐교 되지 않은 학교들 있기 때문에, 폐교기준을 전교생 수로만 고려한다.

In [20]:
#4)전국(전체) 출생아 데이터 & 각학교 전교생 데이터를 통해 상관계수 파악

In [21]:
df_birth.head()

,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,...,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030
시군구별,,,,,,,,,,,,,,,,,,,,,
전국,3634590,3454232,3289596,3107635,2919305,2856193,2825174,2774987,2768200,2800758,...,2742725,2736296,2672368,2558874,2401146,2267367,2104269,1926411,1769354,1641611


In [22]:
pivot_students.head()

조사년도                           2009    2010    2011    2012    2013    2014  \
시도명 학제유형명 개방ID       학교명                                                      
강원  초등학교  1008023023 섬강초등학교     0.0     0.0     0.0     0.0     0.0     0.0   
          1059736549 화동초등학교    58.0    54.0    50.0    44.0    38.0    32.0   
          1110422253 정라초등학교  1269.0  1257.0  1213.0  1164.0  1078.0  1024.0   
          1112041771 성림초등학교  1292.0  1314.0  1311.0  1349.0  1299.0  1267.0   
          1128984749 속사초등학교    20.0    21.0    23.0    22.0    25.0    24.0   

조사년도                           2015    2016    2017    2018    2019    2020  \
시도명 학제유형명 개방ID       학교명                                                      
강원  초등학교  1008023023 섬강초등학교     0.0     0.0     0.0     0.0   956.0  1568.0   
          1059736549 화동초등학교    34.0    43.0    44.0    44.0    41.0    35.0   
          1110422253 정라초등학교   959.0   898.0   840.0   759.0   709.0   678.0   
          1112041771 성림초등학교  1202.0  1151.0  1146.0  1115.0  1141.0  1152.0   
          1128984749 속사초등학교    21.0    27.0    34.0    35.0    49.0    45.0   

조사년도                           2021    2022    2023  
시도명 학제유형명 개방ID       학교명                             
강원  초등학교  1008023023 섬강초등학교  1242.0  1299.0  1360.0  
          1059736549 화동초등학교    31.0    24.0    18.0  
          1110422253 정라초등학교   630.0   567.0   512.0  
          1112041771 성림초등학교  1029.0   942.0   892.0  
          1128984749 속사초등학교    50.0    48.0    37.0

In [23]:
#전국 연도별 전교생 합을 df_students_num 생성
# 2009~2023년 컬럼 리스트 생성
years = [year for year in range(2009, 2024)]

# 연도별 전교생 수를 전체 합산
df_students_num = pd.DataFrame(pivot_students[years].sum(axis=0)).T

# 인덱스 설정
df_students_num.index = ['전국']

In [24]:
df_students_num.head()

조사년도,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
전국,3445343.0,3274312.0,3110552.0,2932419.0,2771949.0,2717509.0,2705546.0,2664881.0,2667577.0,2706564.0,2743590.0,2691366.0,2667376.0,2657386.0,2593420.0


In [25]:
#회귀분석
from scipy.stats import linregress

# X_train (2009~2023)와 Y_train (2009~2023) 설정
X_train = df_birth.loc[:, "2009":"2023"].values.flatten()
Y_train = df_students_num.loc[:, "2009":"2023"].values.flatten()

# 선형 회귀 모델 학습
slope, intercept, r_value, p_value, std_err = linregress(X_train, Y_train)

# 2024~2030년 데이터를 사용하여 Y 예측
X_future = df_birth.loc[:, "2024":"2030"].values.flatten()
Y_pred = intercept + slope * X_future

# 결과 저장
df_reg_results = pd.DataFrame([Y_pred], columns=[str(year) for year in range(2024, 2031)], index=["Predicted_Y"])
df_reg_results["p-value"] = p_value
df_reg_results["R"] = r_value

# DataFrame 저장
All_reg = df_reg_results

In [26]:
All_reg

,2024,2025,2026,2027,2028,2029,2030,p-value,R
Predicted_Y,2.486189e+06,2.348802e+06,2.232275e+06,2.090211e+06,1.935290e+06,1.798488e+06,1.687219e+06,5.149801e-16,0.997185


In [27]:
#시각화
import matplotlib.pyplot as plt

# df_students_num과 All_reg 병합
df_predicted_students = df_students_num
df_predicted_students.loc["전국", "2024":"2030"] = All_reg.loc["Predicted_Y", "2024":"2030"].values

# 그래프 그리기
plt.figure(figsize=(12, 6))

# 실제 학생 수 데이터 (df_students_num + 예측값)
plt.plot(df_predicted_students.columns, df_predicted_students.loc["전국"], marker='o', linestyle='-', label='Predicted Students', color='blue')

# 출생아 수 데이터 (df_birth)
plt.plot(df_birth.columns, df_birth.loc["전국"], marker='s', linestyle='--', label='Births', color='red')

# p-value와 R_squared 표시
p_value = All_reg["p-value"].iloc[0]
r_squared = All_reg["R"].iloc[0]
plt.text(2010, max(df_predicted_students.loc["전국"]) * 0.9, f'p-value: {p_value:.4f}\nR-squared: {r_squared:.4f}', fontsize=12, bbox=dict(facecolor='white', alpha=0.5))

# 그래프 설정
plt.xlabel("Year")
plt.ylabel("Number of Students")
plt.title("Predicted Students vs Births")
plt.legend()
plt.grid()
plt.show()


ValueError: setting an array element with a sequence.

In [ ]:
df_predicted_students

In [ ]:
#5)각지역 출생아 데이터 & 각학교 전교생 데이터를 통해 상관계수 파악

In [ ]:
#연도별 각 전체 학생수 상관관계 분석


#2. 지역 Merge


In [28]:
file_path = "/content/drive/MyDrive/Colab Notebooks/DSL/EDA2_Education/data/재단법인한국지방교육행정연구재단_초중등학교위치.csv"
df_local = pd.read_csv(file_path, encoding='utf-8-sig')

In [29]:
df_local.head()

,학교ID,학교명,학교급구분,설립일자,설립형태,본교분교구분,운영상태,소재지지번주소,소재지도로명주소,시도교육청코드,시도교육청명,교육지원청코드,교육지원청명,생성일자,변경일자,위도,경도,데이터기준일자
0,B000011746,한국산림과학고등학교,고등학교,1972-01-14,공립,본교,운영,경상북도 봉화군 춘양면 서동리 91,경상북도 봉화군 춘양면 서원촌길 8-14,8750000,경상북도교육청,8961000,경상북도봉화교육지원청,2013-11-29,2024-07-09,36.947863,128.914111,2024-09-20
1,B000008267,방이중학교,중학교,1985-04-12,공립,본교,운영,서울특별시 송파구 방이동 53,서울특별시 송파구 오금로11길 64,7010000,서울특별시교육청,7130000,서울특별시강동송파교육지원청,2013-11-29,2024-07-09,37.515011,127.113244,2024-09-20
2,B000009672,율곡중학교,중학교,1955-05-03,사립,본교,운영,경기도 파주시 법원읍 법원리 378,경기도 파주시 법원읍 사임당로905번길 10,7530000,경기도교육청,7681000,경기도파주교육지원청,2013-11-29,2024-07-09,37.849383,126.881141,2024-09-20
3,B000022493,거제상동초등학교,초등학교,2015-05-01,공립,본교,운영,경상남도 거제시 상동동 54,경상남도 거제시 상동5길 75-41,9010000,경상남도교육청,9111000,경상남도거제교육지원청,2013-11-29,2024-07-09,34.861886,128.633904,2024-09-20
4,B000009679,어유중학교,중학교,1979-10-28,공립,본교,운영,경기도 파주시 적성면 어유지리 306,경기도 파주시 적성면 율곡로 2875,7530000,경기도교육청,7681000,경기도파주교육지원청,2013-11-29,2024-07-09,37.993927,126.988278,2024-09-20


In [30]:
pivot_students.head()

조사년도                           2009    2010    2011    2012    2013    2014  \
시도명 학제유형명 개방ID       학교명                                                      
강원  초등학교  1008023023 섬강초등학교     0.0     0.0     0.0     0.0     0.0     0.0   
          1059736549 화동초등학교    58.0    54.0    50.0    44.0    38.0    32.0   
          1110422253 정라초등학교  1269.0  1257.0  1213.0  1164.0  1078.0  1024.0   
          1112041771 성림초등학교  1292.0  1314.0  1311.0  1349.0  1299.0  1267.0   
          1128984749 속사초등학교    20.0    21.0    23.0    22.0    25.0    24.0   

조사년도                           2015    2016    2017    2018    2019    2020  \
시도명 학제유형명 개방ID       학교명                                                      
강원  초등학교  1008023023 섬강초등학교     0.0     0.0     0.0     0.0   956.0  1568.0   
          1059736549 화동초등학교    34.0    43.0    44.0    44.0    41.0    35.0   
          1110422253 정라초등학교   959.0   898.0   840.0   759.0   709.0   678.0   
          1112041771 성림초등학교  1202.0  1151.0  1146.0  1115.0  1141.0  1152.0   
          1128984749 속사초등학교    21.0    27.0    34.0    35.0    49.0    45.0   

조사년도                           2021    2022    2023  
시도명 학제유형명 개방ID       학교명                             
강원  초등학교  1008023023 섬강초등학교  1242.0  1299.0  1360.0  
          1059736549 화동초등학교    31.0    24.0    18.0  
          1110422253 정라초등학교   630.0   567.0   512.0  
          1112041771 성림초등학교  1029.0   942.0   892.0  
          1128984749 속사초등학교    50.0    48.0    37.0

In [31]:
df_local = df_local[df_local['학교급구분']=='초등학교']
df_local = df_local[df_local['운영상태']=='운영']
columns_to_keep = ["학교ID", "학교명","설립일자", "소재지지번주소", "소재지도로명주소", "시도교육청명", "위도", "경도"]
df_local = df_local[columns_to_keep]

In [32]:
df_local.head()

,학교ID,학교명,설립일자,소재지지번주소,소재지도로명주소,시도교육청명,위도,경도
3,B000022493,거제상동초등학교,2015-05-01,경상남도 거제시 상동동 54,경상남도 거제시 상동5길 75-41,경상남도교육청,34.861886,128.633904
5,B000003977,용인한빛초등학교,2010-06-01,경기도 용인시 수지구 동천동 933,경기도 용인시 수지구 동천로 153번길 23,경기도교육청,37.335065,127.087935
6,B000022742,달산초등학교,2014-05-01,부산광역시 기장군 정관읍 달산리 1204,부산광역시 기장군 정관읍 정관로 674,부산광역시교육청,35.319285,129.186736
7,B000003190,삼산초등학교,1931-10-29,인천광역시 강화군 삼산면 석모리 243,인천광역시 강화군 삼산면 삼산북로 451,인천광역시교육청,37.703046,126.321616
8,B000007430,노음초등학교,1936-09-28,경상북도 울진군 근남면 노음리 586,경상북도 울진군 근남면 노음6길 22,경상북도교육청,36.958739,129.391666


In [33]:
# 학교명과 시도교육청명이 모두 같은 학교 찾기
duplicate_rows = df_local[df_local.duplicated(subset=["학교명", "시도교육청명"], keep=False)]

# 중복된 학교 개수 확인
duplicate_count = len(duplicate_rows)

In [34]:
duplicate_rows

,학교ID,학교명,설립일자,소재지지번주소,소재지도로명주소,시도교육청명,위도,경도
148,B000004866,반곡초등학교,2008-03-01,강원특별자치도 원주시 반곡동 1450-1번지,강원특별자치도 원주시 동부순환로 247,강원특별자치도교육청,37.337342,127.973035
226,B000005016,신동초등학교,1950-05-06,강원특별자치도 삼척시 신기면 신기리 356-4 번지,강원특별자치도 삼척시 신기면 강원남부로 2997,강원특별자치도교육청,37.350385,129.081929
675,B000007528,삼계초등학교,1967-03-02,경상남도 창원시 마산회원구 내서읍 삼계리 665,경상남도 창원시 마산회원구 내서읍 삼계로 36,경상남도교육청,35.232669,128.500433
690,B000007218,신기초등학교,1957-02-15,경상북도 문경시 신기동 67-3,경상북도 문경시 신기4길 15,경상북도교육청,36.640048,128.168488
703,B000007514,덕산초등학교,1940-04-24,경상남도 창원시 진해구 자은동 863-1,경상남도 창원시 진해구 덕산로46번길 15,경상남도교육청,35.145879,128.705248
...,...,...,...,...,...,...,...,...
11462,B000007912,신안초등학교,1929-09-04,경상남도 산청군 신안면 하정리 721,경상남도 산청군 신안면 원지로4번길 18,경상남도교육청,35.301903,127.972208
11629,B000007727,상남초등학교,1928-05-27,경상남도 밀양시 상남면 동산리 713-3,경상남도 밀양시 상남면 상남인산길 11,경상남도교육청,35.393382,128.777169
11646,B000004916,중앙초등학교,1957-04-08,강원특별자치도 강릉시 입암동 13-1,강원특별자치도 강릉시 성덕포남로 18,강원특별자치도교육청,37.758087,128.922852
11774,B000007590,금산초등학교,1922-10-09,경상남도 진주시 금산면 중천리 251,경상남도 진주시 금산면 금산로 75,경상남도교육청,35.215303,128.144322


In [35]:
file_path = "/content/drive/MyDrive/Colab Notebooks/DSL/EDA2_Education/data/공시공통사항_학교현황(시도교육청명).csv"
df_sido = pd.read_csv(file_path, encoding='euc-kr')

In [36]:
df_sido = df_sido.drop_duplicates(subset=["개방ID"])
df_sido= df_sido[["개방ID","시도교육청명"]]
df_schools = pd.DataFrame(pivot_students.to_records())
df_schools=df_schools.merge(df_sido, on="개방ID", how="inner")
df_schools.insert(1, "시도교육청명", df_schools.pop("시도교육청명"))
del df_schools["학제유형명"]

In [37]:
#시도교육청명 & 학교명 중복되는 것 행들 제외후 병합 -> 병합안되는 것들 찾아보고 -> 중복되는 것들은 노가다

In [38]:
df_schools

,시도명,시도교육청명,개방ID,학교명,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,강원,강원특별자치도교육청,1008023023,섬강초등학교,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,956.0,1568.0,1242.0,1299.0,1360.0
1,강원,강원특별자치도교육청,1059736549,화동초등학교,58.0,54.0,50.0,44.0,38.0,32.0,34.0,43.0,44.0,44.0,41.0,35.0,31.0,24.0,18.0
2,강원,강원특별자치도교육청,1110422253,정라초등학교,1269.0,1257.0,1213.0,1164.0,1078.0,1024.0,959.0,898.0,840.0,759.0,709.0,678.0,630.0,567.0,512.0
3,강원,강원특별자치도교육청,1112041771,성림초등학교,1292.0,1314.0,1311.0,1349.0,1299.0,1267.0,1202.0,1151.0,1146.0,1115.0,1141.0,1152.0,1029.0,942.0,892.0
4,강원,강원특별자치도교육청,1128984749,속사초등학교,20.0,21.0,23.0,22.0,25.0,24.0,21.0,27.0,34.0,35.0,49.0,45.0,50.0,48.0,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6283,충북,충청북도교육청,9767946541,삼성초등학교,403.0,345.0,307.0,294.0,277.0,258.0,236.0,231.0,220.0,204.0,177.0,164.0,154.0,149.0,139.0
6284,충북,충청북도교육청,9770097672,오석초등학교,66.0,62.0,61.0,80.0,70.0,64.0,59.0,56.0,56.0,60.0,61.0,50.0,35.0,31.0,32.0
6285,충북,충청북도교육청,9819334983,옥산초등학교,777.0,726.0,665.0,607.0,556.0,530.0,518.0,497.0,575.0,595.0,473.0,378.0,340.0,338.0,306.0
6286,충북,충청북도교육청,9872796773,분평초등학교,1054.0,986.0,915.0,823.0,729.0,698.0,661.0,629.0,603.0,569.0,530.0,498.0,481.0,466.0,430.0


In [39]:
unique_rows_school = df_schools[~df_schools.duplicated(subset=['학교명', '시도교육청명'], keep=False)]
unique_rows_school

,시도명,시도교육청명,개방ID,학교명,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,강원,강원특별자치도교육청,1008023023,섬강초등학교,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,956.0,1568.0,1242.0,1299.0,1360.0
1,강원,강원특별자치도교육청,1059736549,화동초등학교,58.0,54.0,50.0,44.0,38.0,32.0,34.0,43.0,44.0,44.0,41.0,35.0,31.0,24.0,18.0
2,강원,강원특별자치도교육청,1110422253,정라초등학교,1269.0,1257.0,1213.0,1164.0,1078.0,1024.0,959.0,898.0,840.0,759.0,709.0,678.0,630.0,567.0,512.0
3,강원,강원특별자치도교육청,1112041771,성림초등학교,1292.0,1314.0,1311.0,1349.0,1299.0,1267.0,1202.0,1151.0,1146.0,1115.0,1141.0,1152.0,1029.0,942.0,892.0
4,강원,강원특별자치도교육청,1128984749,속사초등학교,20.0,21.0,23.0,22.0,25.0,24.0,21.0,27.0,34.0,35.0,49.0,45.0,50.0,48.0,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6283,충북,충청북도교육청,9767946541,삼성초등학교,403.0,345.0,307.0,294.0,277.0,258.0,236.0,231.0,220.0,204.0,177.0,164.0,154.0,149.0,139.0
6284,충북,충청북도교육청,9770097672,오석초등학교,66.0,62.0,61.0,80.0,70.0,64.0,59.0,56.0,56.0,60.0,61.0,50.0,35.0,31.0,32.0
6285,충북,충청북도교육청,9819334983,옥산초등학교,777.0,726.0,665.0,607.0,556.0,530.0,518.0,497.0,575.0,595.0,473.0,378.0,340.0,338.0,306.0
6286,충북,충청북도교육청,9872796773,분평초등학교,1054.0,986.0,915.0,823.0,729.0,698.0,661.0,629.0,603.0,569.0,530.0,498.0,481.0,466.0,430.0


In [40]:
unique_rows_local = df_local[~df_local.duplicated(subset=['학교명', '시도교육청명'], keep=False)]
unique_rows_local

,학교ID,학교명,설립일자,소재지지번주소,소재지도로명주소,시도교육청명,위도,경도
3,B000022493,거제상동초등학교,2015-05-01,경상남도 거제시 상동동 54,경상남도 거제시 상동5길 75-41,경상남도교육청,34.861886,128.633904
5,B000003977,용인한빛초등학교,2010-06-01,경기도 용인시 수지구 동천동 933,경기도 용인시 수지구 동천로 153번길 23,경기도교육청,37.335065,127.087935
6,B000022742,달산초등학교,2014-05-01,부산광역시 기장군 정관읍 달산리 1204,부산광역시 기장군 정관읍 정관로 674,부산광역시교육청,35.319285,129.186736
7,B000003190,삼산초등학교,1931-10-29,인천광역시 강화군 삼산면 석모리 243,인천광역시 강화군 삼산면 삼산북로 451,인천광역시교육청,37.703046,126.321616
8,B000007430,노음초등학교,1936-09-28,경상북도 울진군 근남면 노음리 586,경상북도 울진군 근남면 노음6길 22,경상북도교육청,36.958739,129.391666
...,...,...,...,...,...,...,...,...
11980,B000007762,일운초등학교,1937-04-13,경상남도 거제시 일운면 지세포리 745-1,경상남도 거제시 일운면 지세포로 74,경상남도교육청,34.828474,128.705368
11981,B000007387,예천동부초등학교,1943-06-03,경상북도 예천군 예천읍 백전리 88,경상북도 예천군 예천읍 효자로 171,경상북도교육청,36.661822,128.459064
11983,B000008052,중문초등학교,1927-05-03,제주특별자치도 서귀포시 중문동 1984,제주특별자치도 서귀포시 천제연로 231,제주특별자치도교육청,33.253023,126.430160
11987,B000030436,새봄초등학교,2021-03-01,경기도 화성시 병점동 756-298,경기도 화성시 병점노을로 19,경기도교육청,37.201463,127.029773


In [41]:
df_school_local = pd.merge(unique_rows_school, unique_rows_local, on=['시도교육청명', '학교명'], how='inner')
df_school_local.insert(0, "개방ID", df_school_local.pop("개방ID"))
df_school_local.insert(0, "학교ID", df_school_local.pop("학교ID"))
#df_school_local : 6124

In [42]:
df_school_local

,학교ID,개방ID,시도명,시도교육청명,학교명,2009,2010,2011,2012,2013,...,2019,2020,2021,2022,2023,설립일자,소재지지번주소,소재지도로명주소,위도,경도
0,B000023980,1008023023,강원,강원특별자치도교육청,섬강초등학교,0.0,0.0,0.0,0.0,0.0,...,956.0,1568.0,1242.0,1299.0,1360.0,2019-05-01,강원특별자치도 원주시 지정면 가곡리 1527,강원특별자치도 원주시 지정면 가곡로 91,37.378911,127.874044
1,B000005143,1059736549,강원,강원특별자치도교육청,화동초등학교,58.0,54.0,50.0,44.0,38.0,...,41.0,35.0,31.0,24.0,18.0,1923-07-31,강원특별자치도 정선군 화암면 화암리 380 번지,강원특별자치도 정선군 화암면 그림바위길 66-20,37.338696,128.787955
2,B000005010,1110422253,강원,강원특별자치도교육청,정라초등학교,1269.0,1257.0,1213.0,1164.0,1078.0,...,709.0,678.0,630.0,567.0,512.0,1941-06-07,강원특별자치도 삼척시 정상동 344,강원특별자치도 삼척시 정상로3길 5,37.444291,129.176881
3,B000004834,1112041771,강원,강원특별자치도교육청,성림초등학교,1292.0,1314.0,1311.0,1349.0,1299.0,...,1141.0,1152.0,1029.0,942.0,892.0,2002-09-01,강원특별자치도 춘천시 석사동 868번지,강원특별자치도 춘천시 지석로 47,37.852320,127.740349
4,B000005125,1128984749,강원,강원특별자치도교육청,속사초등학교,20.0,21.0,23.0,22.0,25.0,...,49.0,45.0,50.0,48.0,37.0,1942-07-15,강원특별자치도 평창군 용평면 속사리 352번지,강원특별자치도 평창군 용평면 운두령로 100,37.646460,128.492605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6119,B000005485,9767946541,충북,충청북도교육청,삼성초등학교,403.0,345.0,307.0,294.0,277.0,...,177.0,164.0,154.0,149.0,139.0,1920-06-21,충청북도 음성군 삼성면 덕정리 508-1,충청북도 음성군 삼성면 덕정로 41,37.020068,127.497504
6120,B000005315,9770097672,충북,충청북도교육청,오석초등학교,66.0,62.0,61.0,80.0,70.0,...,61.0,50.0,35.0,31.0,32.0,1952-03-24,충청북도 충주시 금가면 유송리 226-3,충청북도 충주시 금가면 김생로 456,36.999511,127.896058
6121,B000005378,9819334983,충북,충청북도교육청,옥산초등학교,777.0,726.0,665.0,607.0,556.0,...,473.0,378.0,340.0,338.0,306.0,1920-04-14,충청북도 청주시 흥덕구 옥산면 오산리 223,충청북도 청주시 흥덕구 옥산면 옥산시내1길 19,36.663320,127.372842
6122,B000005282,9872796773,충북,충청북도교육청,분평초등학교,1054.0,986.0,915.0,823.0,729.0,...,530.0,498.0,481.0,466.0,430.0,1997-09-01,충청북도 청주시 서원구 분평동 1198,충청북도 청주시 서원구 무심서로 205,36.613578,127.493065


In [43]:
#이름이 달라 빠진 학교들 추가(correct1)
filtered_df = unique_rows_school[~unique_rows_school['개방ID'].isin(df_school_local['개방ID'])]
filtered_df.to_csv("빠진학교_1.csv", index=False, encoding="utf-8-sig")

file_path = "/content/drive/MyDrive/Colab Notebooks/DSL/EDA2_Education/data/빠진학교_1_수정.csv"
df_correct1 = pd.read_csv(file_path, encoding='utf-8-sig')

df_correct1 = df_correct1[df_correct1['폐교여부']=='O']
del df_correct1['폐교여부']

In [44]:
df_correct1_merge = pd.merge(df_correct1, unique_rows_local, on=['시도교육청명', '학교명'], how='inner')

In [45]:
df_correct1_merge.insert(0, "개방ID", df_correct1_merge.pop("개방ID"))
df_correct1_merge.insert(0, "학교ID", df_correct1_merge.pop("학교ID"))

In [46]:
df_correct1_merge

,학교ID,개방ID,시도명,시도교육청명,학교명,2009,2010,2011,2012,2013,...,2019,2020,2021,2022,2023,설립일자,소재지지번주소,소재지도로명주소,위도,경도
0,B000004822,9593145906,강원,강원특별자치도교육청,춘천교육대학교부설초등학교,498,485,487,484,475,...,433,437,422,405,390,1939-05-01,강원특별자치도 춘천시 효자동 420번지,강원특별자치도 춘천시 공지로 316,37.869350,127.731405
1,B000007800,7579956531,경남,경상남도교육청,칠서초등학교이룡분교장,19,12,18,12,7,...,7,7,4,4,2,1943-04-14,경상남도 함안군 칠서면 이룡리 900,경상남도 함안군 칠서면 삼칠로 1335,35.387599,128.505642
2,B000007554,9605033993,경남,경상남도교육청,진주교육대학교부설초등학교,737,720,701,671,640,...,573,581,558,539,523,1964-03-08,경상남도 진주시 신안동 380,경상남도 진주시 진양호로369번길 3,35.185825,128.067928
3,B000022630,6081917047,경북,경상북도교육청,대구교육대학교안동부설초등학교,535,529,509,479,475,...,429,426,417,409,396,1956-04-01,경상북도 안동시 명륜동 산156-5,경상북도 안동시 단원로 222,36.569823,128.728641
4,B000007260,8748309612,경북,대구광역시교육청,대구군위초등학교,411,359,307,291,274,...,315,314,284,292,267,1907-09-01,대구광역시 군위군 군위읍 동부리 323-2,대구광역시 군위군 군위읍 동서5길 28,36.239130,128.568203
5,B000003209,7784405767,광주,광주광역시교육청,광주교육대학교광주부설초등학교,578,581,584,584,583,...,593,576,577,581,580,1938-03-31,광주광역시 북구 풍향동 1-1,광주광역시 북구 필문대로 55,35.164715,126.925697
6,B000024072,1710565984,대구,대구광역시교육청,경북대학교사범대학부설초등학교,771,771,772,741,706,...,584,581,556,537,519,1937-04-01,대구광역시 중구 봉산동 162,대구광역시 중구 달구벌대로 2150,35.863584,128.598066
7,B000002872,2352812886,대구,대구광역시교육청,대구교육대학교대구부설초등학교,528,505,484,460,436,...,435,433,437,425,412,1956-04-01,대구광역시 달서구 도원동 1441-1,대구광역시 달서구 도원남로 19,35.805969,128.532385
8,B000002593,5035131335,부산,부산광역시교육청,부산교육대학교부설초등학교,767,768,752,736,720,...,656,640,624,609,592,1960-05-16,부산광역시 연제구 거제동 331,부산광역시 연제구 교대로 24,35.195693,129.077082
9,B000003106,7857741653,인천,인천광역시교육청,경인교육대학교부설초등학교,672,671,671,668,666,...,575,570,579,571,572,1957-04-10,인천광역시 계양구 효성동 산19-1,인천광역시 계양구 안남로 612,37.537643,126.712507


In [47]:
#중복된 학교 따로 추가(correct2)
duplicate_rows_school = df_schools[df_schools.duplicated(subset=['시도교육청명', '학교명'], keep=False)]
duplicate_rows_local = df_local[df_local.duplicated(subset=['시도교육청명', '학교명'], keep=False)]

duplicate_rows_school = duplicate_rows_school[duplicate_rows_school['2023']>=30]

duplicate_rows_school = duplicate_rows_school.sort_values(by='학교명')
duplicate_rows_local = duplicate_rows_local.sort_values(by='학교명')

duplicate_rows_school.to_csv("빠진학교_2.csv", index=False, encoding="utf-8-sig")
duplicate_rows_local.to_csv("빠진학교_2_위치.csv", index=False, encoding="utf-8-sig")

In [48]:
file_path = "/content/drive/MyDrive/Colab Notebooks/DSL/EDA2_Education/data/빠진학교_2_수정.csv"
df_correct2 = pd.read_csv(file_path, encoding='utf-8-sig')

In [49]:
df_correct2_merge = pd.merge(df_correct2, duplicate_rows_local, on=['시도교육청명', '학교명','학교ID'], how='inner')

In [50]:
#병합(df_school_local에 correct1, correct2 합침)
df_school_local = pd.concat([df_school_local, df_correct1_merge], ignore_index=True)
df_school_local = pd.concat([df_school_local, df_correct2_merge], ignore_index=True)

In [51]:
df_school_local.to_csv("초등학교_정보.csv", index=False, encoding="utf-8-sig")

# 3. 지역별로 출생아수, 전교생수 상관관계 파악

In [52]:
file_path = "/content/drive/MyDrive/Colab Notebooks/DSL/EDA2_Education/data/출생아수_시_군_구.csv"
df_bir = pd.read_csv(file_path, encoding='utf-8-sig')

In [53]:
df_bir

,시군구별,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,서울특별시 종로구,2168,1972,1941,1939,1683,1453,1404,1296,1143,...,905,906,846,689,644,673,529,528,464,396
1,서울특별시 중구,1467,1306,1239,1625,1577,1409,1203,1260,1096,...,1041,950,959,808,733,783,690,627,587,534
2,서울특별시 용산구,2929,2813,2887,3073,2749,2268,2226,2261,2041,...,1976,1967,1708,1464,1377,1314,1176,1234,1110,1012
3,서울특별시 성동구,4865,4496,4511,4885,4306,3917,3864,3685,3368,...,2654,2753,2531,2462,2346,2191,1945,1833,1690,1483
4,서울특별시 광진구,6433,6318,5801,5732,4780,4201,4239,3908,3397,...,3011,2951,2767,2347,2197,1957,1532,1484,1278,1231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,함양군,428,457,406,446,361,310,299,261,270,...,223,209,178,160,150,136,105,99,76,78
246,거창군,764,740,725,707,577,538,452,488,415,...,376,338,352,294,266,238,238,203,175,211
247,합천군,597,561,549,464,418,357,391,328,292,...,207,169,174,123,125,137,107,86,97,58
248,제주시,6031,5754,5766,6220,5476,4770,5040,4548,4179,...,4381,4364,4228,3819,3745,3419,3075,2890,2766,2516


In [54]:
# '시군구별' 열의 공백 제거
df_bir['시군구별'] = df_bir['시군구별'].str.strip()

# 열 이름을 정수형으로 변환
df_bir.columns = [df_bir.columns[0]] + [int(col) for col in df_bir.columns[1:]]

# 숫자 데이터를 정수형(int)으로 변환
for col in df_bir.columns[1:]:
    df_bir[col] = pd.to_numeric(df_bir[col], errors='coerce').fillna(0).astype(int)

# 출생아수 데이터에서 6개년 합계를 구해 초등학교 학생 수 추정 (올바른 계산 방식 적용)
# 2009년부터 2030년까지의 중학교 예상 학생 수 계산
df_ems = pd.DataFrame({'시군구별': df_bir['시군구별']})

for year in range(2009, 2031):  # 2009년부터 2030년까지 추정
    required_years = [year - 12, year-11, year-10, year-9, year-8, year-7]   # 해당 연도에 중학생이 되는 출생연도
    valid_years = [y for y in required_years if y in df_bir.columns]  # 실제 데이터에 존재하는 연도만 사용

    # 출생아 수 합산 (해당 연도가 존재하지 않는 경우 0으로 처리)
    df_ems[f'{year}_해당지역예측학생수'] = df_bir[valid_years].sum(axis=1, numeric_only=True) if valid_years else 0

# 0이 포함된 행이 있을 경우 마지막 0이 위치한 이후 연도의 첫 번째~ 5 번째 셀을 0으로 설정
for idx, row in df_ems.iloc[:, 1:].iterrows():
    zero_indices = row[row == 0].index
    if len(zero_indices) > 0:
        last_zero_idx = zero_indices[-1]
        next_columns = df_ems.columns[df_ems.columns.get_loc(last_zero_idx) + 1:]
        for i in range(min(5, len(next_columns))):
            df_ems.loc[idx, next_columns[i]] = 0

df_ems

,시군구별,2009_해당지역예측학생수,2010_해당지역예측학생수,2011_해당지역예측학생수,2012_해당지역예측학생수,2013_해당지역예측학생수,2014_해당지역예측학생수,2015_해당지역예측학생수,2016_해당지역예측학생수,2017_해당지역예측학생수,...,2021_해당지역예측학생수,2022_해당지역예측학생수,2023_해당지역예측학생수,2024_해당지역예측학생수,2025_해당지역예측학생수,2026_해당지역예측학생수,2027_해당지역예측학생수,2028_해당지역예측학생수,2029_해당지역예측학생수,2030_해당지역예측학생수
0,서울특별시 종로구,11156,10392,9716,8918,8118,7639,7362,7053,6864,...,6183,5994,5733,5319,4863,4663,4287,3909,3527,3234
1,서울특별시 중구,8623,8359,8313,8170,7667,7366,7098,6947,6857,...,6605,6503,6292,5995,5513,5274,4923,4600,4228,3954
2,서울특별시 용산구,16719,16016,15464,14618,13706,13406,13470,13403,13454,...,13296,13104,12500,11655,10621,9806,9006,8273,7675,7223
3,서울특별시 성동구,26980,25979,25168,24025,22599,22055,21510,20469,19670,...,16812,16742,16387,16073,15584,14937,14228,13308,12467,11488
4,서울특별시 광진구,33265,31071,28661,26257,24091,23025,22303,21322,20780,...,19128,18821,18222,17385,16263,15230,13751,12284,10795,9679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,함양군,2408,2279,2083,1947,1799,1750,1717,1680,1695,...,1458,1405,1307,1245,1158,1056,938,828,726,644
246,거창군,4051,3739,3487,3177,2872,2772,2694,2649,2582,...,2454,2385,2316,2168,2019,1864,1726,1591,1414,1331
247,합천군,2946,2740,2507,2250,2055,1979,1955,1870,1799,...,1532,1395,1312,1151,1031,935,835,752,675,610
248,제주시,34017,33026,31820,30233,28254,27300,26620,25582,25283,...,25752,26114,26093,25655,24728,23956,22650,21176,19714,18411


In [55]:
# '소재지지번주소'가 NA가 아닌 데이터만 사용
df_school_local = df_school_local.dropna(subset=['소재지지번주소'])

# '소재지지번주소'에서 '시군구별' 값이 포함된 경우 매핑
def find_region(address):
    for region in df_ems['시군구별']:
        if region in address:
            return region
    return None

# 학생 수 데이터에 '출생아수' 정보를 추가
df_school_local['시군구별'] = df_school_local['소재지지번주소'].apply(find_region)

# 매칭된 데이터와 출생아수 데이터 병합
df_s_l = df_school_local.merge(df_ems, on='시군구별', how='left')

# 예측학생 수 정수 변환
#df_s_l.iloc[:, 24:] = df_s_l.iloc[:, 24:].fillna(0).astype(int)


In [56]:
df_s_l

,학교ID,개방ID,시도명,시도교육청명,학교명,2009,2010,2011,2012,2013,...,2021_해당지역예측학생수,2022_해당지역예측학생수,2023_해당지역예측학생수,2024_해당지역예측학생수,2025_해당지역예측학생수,2026_해당지역예측학생수,2027_해당지역예측학생수,2028_해당지역예측학생수,2029_해당지역예측학생수,2030_해당지역예측학생수
0,B000023980,1008023023,강원,강원특별자치도교육청,섬강초등학교,0.0,0.0,0.0,0.0,0.0,...,16782,16521,16050,15398,14460,14060,13627,13057,12660,12303
1,B000005143,1059736549,강원,강원특별자치도교육청,화동초등학교,58.0,54.0,50.0,44.0,38.0,...,1531,1442,1347,1241,1111,1053,945,877,791,737
2,B000005010,1110422253,강원,강원특별자치도교육청,정라초등학교,1269.0,1257.0,1213.0,1164.0,1078.0,...,2792,2748,2651,2515,2334,2264,2204,2070,1984,1923
3,B000004834,1112041771,강원,강원특별자치도교육청,성림초등학교,1292.0,1314.0,1311.0,1349.0,1299.0,...,13276,13220,12937,12275,11675,11214,10764,10194,9710,9298
4,B000005125,1128984749,강원,강원특별자치도교육청,속사초등학교,20.0,21.0,23.0,22.0,25.0,...,1466,1418,1329,1285,1168,1067,974,872,782,689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6238,B000007485,4932731568,경남,경상남도교육청,평산초등학교,1136.0,1056.0,972.0,884.0,812.0,...,0,13827,13577,13153,12396,11758,11007,9876,8901,8041
6239,B000007835,8832080364,경남,경상남도교육청,평산초등학교,828.0,870.0,892.0,858.0,872.0,...,16672,17312,17507,17552,17650,17657,17066,16083,14924,13601
6240,B000007539,7106910347,경남,경상남도교육청,하북초등학교,125.0,116.0,106.0,92.0,85.0,...,0,8743,8600,8222,7629,7187,6521,6012,5552,5136
6241,B000007850,7579760730,경남,경상남도교육청,하북초등학교,440.0,400.0,369.0,323.0,268.0,...,16672,17312,17507,17552,17650,17657,17066,16083,14924,13601


In [57]:
df_s_l.columns

Index(['학교ID', '개방ID', '시도명', '시도교육청명', '학교명', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021',
       '2022', '2023', '설립일자', '소재지지번주소', '소재지도로명주소', '위도', '경도', '시군구별',
       '2009_해당지역예측학생수', '2010_해당지역예측학생수', '2011_해당지역예측학생수', '2012_해당지역예측학생수',
       '2013_해당지역예측학생수', '2014_해당지역예측학생수', '2015_해당지역예측학생수', '2016_해당지역예측학생수',
       '2017_해당지역예측학생수', '2018_해당지역예측학생수', '2019_해당지역예측학생수', '2020_해당지역예측학생수',
       '2021_해당지역예측학생수', '2022_해당지역예측학생수', '2023_해당지역예측학생수', '2024_해당지역예측학생수',
       '2025_해당지역예측학생수', '2026_해당지역예측학생수', '2027_해당지역예측학생수', '2028_해당지역예측학생수',
       '2029_해당지역예측학생수', '2030_해당지역예측학생수'],
      dtype='object')

In [58]:
#열 재정
columns_to_move = ['설립일자', '소재지지번주소', '소재지도로명주소', '위도', '경도', '시군구별']  # 이동할 열 목록
position = 5  # 새로운 위치 (0부터 시작)
remaining_columns = [col for col in df_s_l.columns if col not in columns_to_move]
#새 열 순서 조합
new_column_order = remaining_columns[:position] + columns_to_move + remaining_columns[position:]

# 열 순서 변경
df_s_l = df_s_l[new_column_order]

In [59]:
df_s_l.columns

Index(['학교ID', '개방ID', '시도명', '시도교육청명', '학교명', '설립일자', '소재지지번주소', '소재지도로명주소',
       '위도', '경도', '시군구별', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023',
       '2009_해당지역예측학생수', '2010_해당지역예측학생수', '2011_해당지역예측학생수', '2012_해당지역예측학생수',
       '2013_해당지역예측학생수', '2014_해당지역예측학생수', '2015_해당지역예측학생수', '2016_해당지역예측학생수',
       '2017_해당지역예측학생수', '2018_해당지역예측학생수', '2019_해당지역예측학생수', '2020_해당지역예측학생수',
       '2021_해당지역예측학생수', '2022_해당지역예측학생수', '2023_해당지역예측학생수', '2024_해당지역예측학생수',
       '2025_해당지역예측학생수', '2026_해당지역예측학생수', '2027_해당지역예측학생수', '2028_해당지역예측학생수',
       '2029_해당지역예측학생수', '2030_해당지역예측학생수'],
      dtype='object')

In [60]:
df_s_l.to_csv("elementary_info.csv", index=False, encoding='utf-8-sig')

In [61]:
#회귀분석

from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

In [62]:
# X(해당지역예측학생수), Y(학생수)열 선택
columns_to_check = df_s_l.filter(regex=r'^(2009|201[0-9]|202[0-3])_해당지역예측학생수$').columns.tolist() + \
                   df_s_l.filter(regex=r'^(2009|201[0-9]|202[0-3])$').columns.tolist()

# 0이 포함된 행 삭제
df_nonzero = df_s_l[~(df_s_l[columns_to_check] == 0).any(axis=1)]

In [63]:
df_empty = pd.DataFrame(columns=['시군구별', '상관계수'])
print(df_empty)

Empty DataFrame
Columns: [시군구별, 상관계수]
Index: []


In [64]:
# 시군구별 출생아수와 학생수의 상관관계 분석

for region in df_nonzero['시군구별'].unique() :
    df_temp = df_s_l[df_s_l['시군구별'] == region]
    temp_train_X = df_temp.filter(regex=r'^(2009|201[0-9]|202[0-3])_해당지역예측학생수$').iloc[0].values
    temp_train_y = df_temp.filter(regex=r'^(2009|201[0-9]|202[0-3])$')
    temp_train_y = temp_train_y.sum(axis=0).values

    # 상관계수 계산
    correlation_matrix = np.corrcoef(temp_train_X, temp_train_y)
    df_empty.loc[len(df_empty)] = [region, correlation_matrix[0, 1]]

In [65]:
# X, Y 간 상관계수 열 추가

df_nonzero.loc[:, '상관계수'] = np.nan

for i in df_nonzero.index:
    train_df = df_nonzero.loc[[i]]

    # X_train, y_train 선택
    X_train = train_df.filter(regex=r'^(2009|201[0-9]|202[0-3])_해당지역예측학생수$').values.flatten()
    y_train = train_df.filter(regex=r'^(2009|201[0-9]|202[0-3])$').values.flatten()

    # 상관계수 계산
    correlation_matrix = np.corrcoef(X_train, y_train)
    df_nonzero.loc[i, '상관계수'] = correlation_matrix[0, 1]

<ipython-input-65-64f8ef0635b8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nonzero.loc[:, '상관계수'] = np.nan


In [66]:
# 상관계수 0.8 이상인 지역 학교 filtering
df_positive = df_nonzero[df_nonzero['상관계수'] >= 0.8]
df_positive

,학교ID,개방ID,시도명,시도교육청명,학교명,설립일자,소재지지번주소,소재지도로명주소,위도,경도,...,2022_해당지역예측학생수,2023_해당지역예측학생수,2024_해당지역예측학생수,2025_해당지역예측학생수,2026_해당지역예측학생수,2027_해당지역예측학생수,2028_해당지역예측학생수,2029_해당지역예측학생수,2030_해당지역예측학생수,상관계수
1,B000005143,1059736549,강원,강원특별자치도교육청,화동초등학교,1923-07-31,강원특별자치도 정선군 화암면 화암리 380 번지,강원특별자치도 정선군 화암면 그림바위길 66-20,37.338696,128.787955,...,1442,1347,1241,1111,1053,945,877,791,737,0.860129
2,B000005010,1110422253,강원,강원특별자치도교육청,정라초등학교,1941-06-07,강원특별자치도 삼척시 정상동 344,강원특별자치도 삼척시 정상로3길 5,37.444291,129.176881,...,2748,2651,2515,2334,2264,2204,2070,1984,1923,0.913410
6,B000005227,1154525279,강원,강원특별자치도교육청,대진초등학교,1926-04-01,강원특별자치도 고성군 현내면 대진리 148 번지,강원특별자치도 고성군 현내면 대진3길 15,38.496963,128.423592,...,1041,1018,940,856,821,761,706,668,620,0.934596
8,B000004873,1180781960,강원,강원특별자치도교육청,평원초등학교,1995-03-15,강원특별자치도 원주시 단계동 814 번지,강원특별자치도 원주시 금불2길 20,37.349682,127.931752,...,16521,16050,15398,14460,14060,13627,13057,12660,12303,0.856967
9,B000004836,1202087283,강원,강원특별자치도교육청,남춘천초등학교,1951-10-15,강원특별자치도 춘천시 온의동 550 번지,강원특별자치도 춘천시 충혼길 33,37.861621,127.722052,...,13220,12937,12275,11675,11214,10764,10194,9710,9298,0.886417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6228,B000004982,8561446547,강원,강원특별자치도교육청,조양초등학교,1970-12-31,강원특별자치도 속초시 조양동 1396 번지,강원특별자치도 속초시 동해대로 3965,38.187842,128.598124,...,3862,3679,3395,3166,2984,2790,2525,2377,2302,0.956679
6231,B000004826,1974684715,강원,강원특별자치도교육청,중앙초등학교,1945-10-15,강원특별자치도 춘천시 낙원동 78 번지,강원특별자치도 춘천시 중앙로88번길 14,37.879247,127.723811,...,13220,12937,12275,11675,11214,10764,10194,9710,9298,0.976163
6232,B000004863,6505026936,강원,강원특별자치도교육청,중앙초등학교,1969-03-26,강원특별자치도 원주시 학성동 1260 번지,강원특별자치도 원주시 남산로 181,37.352782,127.939086,...,16521,16050,15398,14460,14060,13627,13057,12660,12303,0.968454
6233,B000004916,5872217655,강원,강원특별자치도교육청,중앙초등학교,1957-04-08,강원특별자치도 강릉시 입암동 13-1,강원특별자치도 강릉시 성덕포남로 18,37.758087,128.922852,...,9108,8703,8116,7516,7020,6672,6162,5819,5561,0.926590


In [67]:
# 2024~2030년의 "예측학생수" 열 추가 (초기값 NaN)
for year in range(2024, 2030):
    df_positive.loc[:,f'{year}_예측학생수'] = np.nan

df_positive.columns

<ipython-input-67-092b01e179bf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_positive.loc[:,f'{year}_예측학생수'] = np.nan
<ipython-input-67-092b01e179bf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_positive.loc[:,f'{year}_예측학생수'] = np.nan
<ipython-input-67-092b01e179bf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

Index(['학교ID', '개방ID', '시도명', '시도교육청명', '학교명', '설립일자', '소재지지번주소', '소재지도로명주소',
       '위도', '경도', '시군구별', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023',
       '2009_해당지역예측학생수', '2010_해당지역예측학생수', '2011_해당지역예측학생수', '2012_해당지역예측학생수',
       '2013_해당지역예측학생수', '2014_해당지역예측학생수', '2015_해당지역예측학생수', '2016_해당지역예측학생수',
       '2017_해당지역예측학생수', '2018_해당지역예측학생수', '2019_해당지역예측학생수', '2020_해당지역예측학생수',
       '2021_해당지역예측학생수', '2022_해당지역예측학생수', '2023_해당지역예측학생수', '2024_해당지역예측학생수',
       '2025_해당지역예측학생수', '2026_해당지역예측학생수', '2027_해당지역예측학생수', '2028_해당지역예측학생수',
       '2029_해당지역예측학생수', '2030_해당지역예측학생수', '상관계수', '2024_예측학생수', '2025_예측학생수',
       '2026_예측학생수', '2027_예측학생수', '2028_예측학생수', '2029_예측학생수'],
      dtype='object')

In [68]:
# 회귀모델 적합 + 예측

for i in df_positive.index:
    train_df = df_positive.loc[[i]]

    # 독립변수(X), 종속변수(y) 선택
    X_train = train_df.filter(regex=r'^(2009|201[0-9]|202[0-3])_해당지역예측학생수$').values.reshape(-1, 1)
    y_train = train_df.filter(regex=r'^(2009|201[0-9]|202[0-3])$').values.reshape(-1, 1)

    # 회귀 모델 학습
    model = LinearRegression()
    model.fit(X_train, y_train)

    # 예측할 연도의 지역 중학생 수 데이터
    X_future = train_df.filter(regex=r'^(202[4-9]|2030)_해당지역예측학생수$').values.reshape(-1, 1)

    # 미래 학생 수 예측
    predictions = model.predict(X_future).flatten()
    df_positive.loc[i, [f"{year}_예측학생수" for year in range(2024, 2031)]] = predictions

In [69]:
df_positive.head()

,학교ID,개방ID,시도명,시도교육청명,학교명,설립일자,소재지지번주소,소재지도로명주소,위도,경도,...,2029_해당지역예측학생수,2030_해당지역예측학생수,상관계수,2024_예측학생수,2025_예측학생수,2026_예측학생수,2027_예측학생수,2028_예측학생수,2029_예측학생수,2030_예측학생수
1,B000005143,1059736549,강원,강원특별자치도교육청,화동초등학교,1923-07-31,강원특별자치도 정선군 화암면 화암리 380 번지,강원특별자치도 정선군 화암면 그림바위길 66-20,37.338696,128.787955,...,791,737,0.860129,24.589641,21.367247,19.929563,17.252497,15.566937,13.435200,12.096667
2,B000005010,1110422253,강원,강원특별자치도교육청,정라초등학교,1941-06-07,강원특별자치도 삼척시 정상동 344,강원특별자치도 삼척시 정상로3길 5,37.444291,129.176881,...,1984,1923,0.913410,631.210445,574.663941,552.795127,534.050430,492.187272,465.319873,446.262764
6,B000005227,1154525279,강원,강원특별자치도교육청,대진초등학교,1926-04-01,강원특별자치도 고성군 현내면 대진리 148 번지,강원특별자치도 고성군 현내면 대진3길 15,38.496963,128.423592,...,668,620,0.934596,24.771214,20.379338,18.549390,15.412335,12.536702,10.549901,8.040258
8,B000004873,1180781960,강원,강원특별자치도교육청,평원초등학교,1995-03-15,강원특별자치도 원주시 단계동 814 번지,강원특별자치도 원주시 금불2길 20,37.349682,127.931752,...,12660,12303,0.856967,683.052825,617.029882,588.875108,558.397566,518.277015,490.333402,465.205267
9,B000004836,1202087283,강원,강원특별자치도교육청,남춘천초등학교,1951-10-15,강원특별자치도 춘천시 온의동 550 번지,강원특별자치도 춘천시 충혼길 33,37.861621,127.722052,...,9710,9298,0.886417,389.038092,340.928542,303.964372,267.882210,222.178138,183.369768,150.334544


In [70]:
df_positive.to_csv("초등학교_예측결과.csv", index=False, encoding='utf-8-sig')

In [71]:
# 2036년 기준 학생수 0명 이하인 학교 filtering
df_danger = df_positive[df_positive['2030_예측학생수'] < 0]

In [72]:
df_save = df_positive[df_positive['2030_예측학생수'] >= 0]

In [73]:
df_danger.to_csv("초등학교_폐교위험.csv", index=False, encoding='utf-8-sig')

In [74]:
df_save.to_csv("초등학교_안전.csv", index=False, encoding='utf-8-sig')